# title placeholder


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


## Data Loading


In [3]:
ebird_data = pd.read_csv("ebird_nm_2022.csv")


In [4]:
ebird_data.head()


,Unnamed: 0,COMMON_NAME,LATITUDE,LONGITUDE,OBSERVATION_DATE
0,0,Mallard/Mexican Duck,35.158855,-106.633675,2015-04-11
1,1,Common Merganser,33.802000,-106.880000,2005-12-27
2,2,Long-billed Dowitcher,33.802000,-106.880000,2005-02-03
3,3,American Avocet,33.802000,-106.880000,2007-04-14
4,4,Great Egret,33.802000,-106.880000,2003-05-22


In [5]:
ebird_data.shape


(9009593, 5)

## Quick Data Cleaning


In [6]:
ebird_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9009593 entries, 0 to 9009592
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Unnamed: 0        int64  
 1   COMMON_NAME       object 
 2   LATITUDE          float64
 3   LONGITUDE         float64
 4   OBSERVATION_DATE  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 343.7+ MB


In [7]:
ebird_data.columns = ebird_data.columns.str.title()()


In [8]:
ebird_data.columns


Index(['unnamed: 0', 'common_name', 'latitude', 'longitude',
       'observation_date'],
      dtype='object')

In [9]:
#drop first column
ebird_data = ebird_data.drop(columns=['Unnamed: 0'])


In [ ]:
# replace underscores with spaces in column names
ebird_data.columns = ebird_data.columns.str.replace('_', ' ')


In [10]:
ebird_data['Observation_date'] = pd.to_datetime(ebird_data['Observation_date'])


In [55]:
def add_datetime_components(df, datetime_column_name) -> None:
    """
    Extracts components of a datetime column into new columns
    
    Args:
        df (pd.DataFrame): 
        datetime_column_name (str): Datetime column name 

    Returns:
        None, modifies the dataframe in place
    """
    # extract datetime components into new columns
    df['Month'] = df[datetime_column_name].dt.month
    df['Year'] = df[datetime_column_name].dt.year
    df['Week Of Year'] = df[datetime_column_name].dt.isocalendar().week
    df['Day'] = df[datetime_column_name].dt.dayofyear
    
    return None


In [ ]:
add_datetime_components(ebird_data, 'Observation Date')


In [12]:
ebird_data.head() 


,common_name,latitude,longitude,observation_date,year,month,day,week of year
0,Mallard/Mexican Duck,35.158855,-106.633675,2015-04-11,2015,4,11,15
1,Common Merganser,33.802000,-106.880000,2005-12-27,2005,12,27,52
2,Long-billed Dowitcher,33.802000,-106.880000,2005-02-03,2005,2,3,5
3,American Avocet,33.802000,-106.880000,2007-04-14,2007,4,14,15
4,Great Egret,33.802000,-106.880000,2003-05-22,2003,5,22,21


In [13]:
ebird_data_copy = ebird_data.copy()


## Functions


### Species Observations Over Time


In [14]:
def species_observations_over_time(data: pd.DataFrame, species: str, time_units: str) -> pd.DataFrame:
    """
    This function takes a dataframe and returns a dataframe with the number of observations of a species over a given unit of time.
    i.e. number of obersvations of the American Crow per month, week, or day.

    Args:
        data (pd.DataFrame): eBird dataframe
        species (str): Bird Species of interest
        time_units (str): Unit of time to aggregate observations. Options: 'Month', 'Week', 'Day'

    Raises:
        ValueError: If time unit is not supported
        ValueError: If species is not found in data

    Returns:
        pd.DataFrame: DataFrame with the number of observations of a species over a given unit of time.
    """
    # Dictionary to map time units to column names
    time_dict = {
        'Month': 'month',
        'Week': 'week of year',
        'Day': 'day',
    }
    
    if time_units not in time_dict:
        raise ValueError(f"Time unit {time_units} not supported")
    
    species_data = data[data['common_name'] == species]
    
    if species_data.empty:
        raise ValueError(f"Species {species} not found in data")
    
    species_observations = species_data.groupby(['Year', time_dict[time_units]]).size().reset_index(name='Observations')
    
    return species_observations


Example Usage


In [16]:
american_crow_monthly = species_observations_over_time(ebird_data, 'American Crow', 'Month')
american_crow_monthly.head(12)


,year,month,observations
0,1990,1,21
1,1990,2,21
2,1990,3,13
3,1990,4,4
4,1990,5,5
5,1990,6,1
6,1990,7,1
7,1990,8,15
8,1990,9,14
9,1990,10,15


In [27]:
american_crow_monthly['year'].unique()


array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
      dtype=int64)

In [57]:
american_crow_weekly = species_observations_over_time(ebird_data, 'American Crow', 'week')
american_crow_weekly.head(12)


,year,week of year,observations
0,1990,1,5
1,1990,2,4
2,1990,3,4
3,1990,4,8
4,1990,5,8
5,1990,6,6
6,1990,7,3
7,1990,8,4
8,1990,9,3
9,1990,11,1


## Polynomial Fit


In [38]:
def calculate_fitted_polynomials(data: pd.DataFrame, time_units: str, degree: int) -> np.ndarray:
    """
    This function takes a dataframe and returns an array of the coefficients of the polynomial that best fits the number of observations of a species over a given unit of time.

    Args:
        data (pd.DataFrame): Aggregate dataframe of species observations over time
        species (str): Species of interest
        time_units (str): Unit of time to aggregate observations. Options: 'Month', 'Week', 'Day'
        degree (int): Degree of the polynomial to fit the data

    Returns:
        np.ndarray: Array of the coefficients of the polynomial that best fits the number of observations of a species over a given unit of time.
    """
    #create an empty numpy array to store the polynomials
    polynomial_array = np.array([])
    #loop through the years in the data and fit a polynomial to the observations
    for year in data['year'].unique():
        species_year = data[data['year'] == year]
        x = species_year[time_units]
        y = species_year['observations']
        poly = np.polyfit(x, y, degree)
        polynomial_array = np.append(polynomial_array, poly)
             
    return polynomial_array
    
    



In [28]:
american_crow_polynomials = calculate_fitted_polynomials(american_crow_monthly, 'month', 5)
len(american_crow_polynomials)


33

## Calculate Inflection Points


In [39]:
def calculate_inflection_points(polynomial: np.array) -> np.ndarray:
    """
    This function takes an array of polynomial coefficients and returns an array of the inflection points of the polynomial.

    Args:
        polynomial (np.array): Array of polynomial coefficients

    Returns:
        np.ndarray: Array of the inflection points of the polynomial.
    """
    #create an empty numpy array to store the inflection points
    inflection_points = np.array([])
    #loop through the array of polynomials and calculate the inflection points
    for poly in polynomial:
        p = np.poly1d(poly)
        second_derivative = np.polyder(p, 2)
        inflection = np.roots(second_derivative)
        inflection_points = np.append(inflection_points, inflection)

    return inflection_points


### Example Usage

## Plot Bird Observations with Polynomial Fit

In [50]:
def plot_obervations_with_polynomial_fit(data: pd.DataFrame, species: str, time_units: str, degree: int) -> None:
    """
    This function takes a dataframe and plots the number of observations of a species over a given unit of time with the polynomial that best fits the data.

    Args:
        data (pd.DataFrame): eBird dataframe
        species (str): Bird Species of interest
        time_units (str): Unit of time to aggregate observations. Options: 'Month', 'Week', 'Day'
        degree (int): Degree of the polynomial to fit the data

    Returns:
        None, but displays a plot
    """
    # get species observations
    species_observations = species_observations_over_time(data, species, time_units)
    # calculate polynomial coefficients
    polynomial_coefficients = calculate_fitted_polynomials(species_observations, time_units, degree)
    # calculate inflection points
    inflection_points = calculate_inflection_points(polynomial_coefficients)
    
    # plot observations
    fig = px.line(species_observations,
                  x=time_units,
                  y='observations',
                  color='year',
                  markers=False,
                  title=f'Observation Count for {species} by {time_units.title()}')
    
    # plot polynomial approximations for each year
    for year in species_observations['year'].unique():
        year_data = species_observations[species_observations['year'] == year]
        coefficients = np.polyfit(year_data[time_units], year_data['observations'], degree)
        polynomial = np.poly1d(coefficients)
        
        fig.add_scatter(x=year_data[time_units],
                        y=polynomial(year_data[time_units]),
                        mode='lines', line=dict(dash='dash'),
                        visible='legendonly',
                        name=f'Polynomial Approximation ({year})'
                        )
    
    # TODO: plot inflection points as vertical dashed lines


    # display plot
    fig.show()
    
    return None
    
   


### Example Usage

In [54]:
plot_obervations_with_polynomial_fit(ebird_data, 'American Crow', 'month', 5)
